### **What is LangChain?**
> LangChain is a framework for developing applications powered by language models.

> LangChain makes the complicated parts of working & building with AI models easier. It helps do this in two ways:

1. **Integration** - Bring external data, such as your files, other applications, and api data, to your LLMs

2. **Agency** - Allow your LLMs to interact with it's environment via decision making. Use LLMs to help decide which action to take next

# Langchain crash course

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key=os.getenv('OPENAI_API_KEY')


## LLMs

In [3]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9, openai_api_key=openai_api_key)
name = llm.predict("I want to open a restaurant for Indian food. Suggest a fency name for this.")
print(name)



Tastes of India


In [4]:
llm("I want to open a restaurant for Indian food. Suggest a fency name for this.")

'\n\nSpice Temple'

In [5]:
# You'll be working with simple strings (that'll soon grow in complexity!)
my_text = "What day comes after Friday?"
my_text

'What day comes after Friday?'

### **Chat Messages**
Like text, but specified with a message type (System, Human, AI)

* **System** - Helpful background context that tell the AI what to do
* **Human** - Messages that are intented to represent the user
* **AI** - Messages that show what the AI responded with

For more, see OpenAI's [documentation](https://platform.openai.com/docs/guides/chat/introduction)

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

# This it the language model we'll use. We'll talk about what we're doing below in the next section
chat = ChatOpenAI(temperature=.7, openai_api_key=openai_api_key)

Now let's create a few messages that simulate a chat experience with a bot

In [8]:
chat(
    [
        SystemMessage(content="You are a nice AI bot that helps a user figure out what to eat in one short sentence"),
        HumanMessage(content="I like tomatoes, what should I eat?")
    ]
)

AIMessage(content='You can try a Caprese salad with fresh tomatoes, mozzarella, and basil.')

You can also pass more chat history w/ responses from the AI

In [9]:
chat(
    [
        SystemMessage(content="You are a nice AI bot that helps a user figure out where to travel in one short sentence"),
        HumanMessage(content="I like the beaches where should I go?"),
        AIMessage(content="You should go to Nice, France"),
        HumanMessage(content="What else should I do when I'm there?")
    ]
)

AIMessage(content='You should explore the charming old town, visit the stunning Promenade des Anglais, and indulge in delicious French cuisine.')

You can also exclude the system message if you want

In [10]:
chat(
    [
        HumanMessage(content="What day comes after Thursday?")
    ]
)

AIMessage(content='Friday comes after Thursday.')

## Prompt Templates

**Zero-Shot Prompting**

In [12]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

I want to open a restaurant for Italian food. Suggest a fency name for this.


In [13]:
llm.predict(p)

'\n\nCucina Deliziosa'

**Few-shots Prompting**

In [15]:
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
]

# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())


Human: 2+2
AI: 4
Human: 2+3
AI: 5


In [17]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [19]:
from langchain.chat_models import ChatOpenAI

chain = final_prompt | ChatOpenAI(temperature=.7, openai_api_key=openai_api_key)

chain.invoke({"input": "What's the square of a triangle?"})

AIMessage(content='A triangle does not have a square. A square is a quadrilateral with four equal sides and four right angles, while a triangle is a polygon with three sides.')

In [20]:
chain.invoke({"input": "What's the area of a triangle?"})

AIMessage(content="The area of a triangle can be calculated using the formula: \n\nArea = (base * height) / 2\n\nwhere the base is the length of the triangle's base and the height is the perpendicular distance from the base to the opposite vertex.")

**Chain of Thoughts**

In [24]:
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

examples = [
    {"input": "The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.", "output": "Adding all the odd numbers (9, 15, 1) gives 25."},
    {"input": "The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.", "output": "Adding all the odd numbers (17, 19) gives 36."},
    {"input": "The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.", "output": "Adding all the odd numbers (11, 13) gives 24."},
    {"input": "The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.", "output": "Adding all the odd numbers (17, 9, 13) gives 39."},
]

# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
AI: Adding all the odd numbers (9, 15, 1) gives 25.
Human: The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
AI: Adding all the odd numbers (17, 19) gives 36.
Human: The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
AI: Adding all the odd numbers (11, 13) gives 24.
Human: The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
AI: Adding all the odd numbers (17, 9, 13) gives 39.


In [25]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [26]:
from langchain.chat_models import ChatOpenAI

chain = final_prompt | ChatOpenAI(temperature=.7, openai_api_key=openai_api_key)

chain.invoke({"input": "The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. "})

AIMessage(content='Adding all the odd numbers (15, 5, 13, 7, 1) gives 41.')

## Chains

In [28]:
from langchain.chains import LLMChain

llm = OpenAI(temperature=1, openai_api_key=openai_api_key)

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Mexican")

'\n\n"Taco Fiesta Palacio".'

In [7]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
chain.run("Mexican")



> Entering new  chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.


' \n\nTaco Tequila Palace'

In [31]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

#### Simple Sequential Chain

In [32]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.run("Indian")
print(content)



-Chickpea Curry
-Tandoori Chicken
-Butter Chicken
-Vegetable Biryani
-Naan Bread
-Aloo Gobi
-Vegetable Samosas
-Paneer Tikka Masala
-Lamb Korma
-Spicy Eggplant Curry
-Dal Makhani
-Coconut Rice
-Mango Lassi


#### Sequential Chain

In [33]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [34]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [35]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [36]:
chain({"cuisine": "Indian"})

{'cuisine': 'Indian',
 'restaurant_name': '\n\nRoyal Spice Cuisine',
 'menu_items': '\n\n- Lamb Biryani\n- Chicken Tikka Masala\n- Paneer Tikka\n- Saag Paneer\n- Tandoori Chicken\n- Aloo Gobi\n- Kashmiri Korma\n- Chicken Korma\n- Shahi Paneer\n- Chicken Vindaloo\n- Vegetable Jalfrezi\n- Palak Paneer\n- Fish Curry\n- Egg Curry'}

## Agents

In [62]:
# make sure yoy have installed this package: pip install google-search-results
# from secret_key import serpapi_key
# os.environ['SERPAPI_API_KEY'] = serpapi_key

os.environ['SERPAPI_API_KEY'] =os.getenv('SERPAPI_API_KEY')

#### serpapi and llm-math tool

In [64]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Let's test it out!
agent.run("What was the GDP of US in 2022 plus 5?")



> Entering new AgentExecutor chain...
 I need to find the GDP of US in 2022
Action: Search
Action Input: US GDP in 2022
Observation: about 25.46 trillion U.S. dollars
Thought: I need to add 5 to this number
Action: Calculator
Action Input: 25.46 + 5
Observation: Answer: 30.46
Thought: I now know the final answer
Final Answer: 30.46 trillion U.S. dollars

> Finished chain.


'30.46 trillion U.S. dollars'

#### Wikipedia and llm-math tool

In [65]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

# Let's test it out!
agent.run("When was Elon musk born? What is his age right now in 2023?")



> Entering new AgentExecutor chain...
 I need to find out when Elon Musk was born and then calculate his age.
Action: Wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman and investor. He is the wealthiest person in the world, with an estimated net worth of US$222 billion as of December 2023, according to the Bloomberg Billionaires Index, and $244 billion according to Forbes, primarily from his ownership stakes in Tesla and SpaceX. He is the founder, chairman, CEO, and chief technology officer of SpaceX; angel investor, CEO, product architect and former chairman of Tesla, Inc.; owner, chairman and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation.
A member of the wealthy South African Musk family, Elon was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizen

'Elon Musk was born on June 28, 1971 and acquired Twitter in October 2022. He is currently 52 years old in 2023.'

## Memory

In [66]:
chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("Mexican")
print(name)



Taco Fiesta Palace


In [67]:
name = chain.run("Indian")
print(name)



Maharaja's Palace Cuisine


In [68]:
chain.memory

In [69]:
type(chain.memory)

NoneType

#### ConversationBufferMemory

In [70]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)



Taco Fiesta Palace


In [71]:
name = chain.run("Arabic")
print(name)



Al-Fez Restaurant


In [72]:
print(chain.memory.buffer)

Human: Mexican
AI: 

Taco Fiesta Palace
Human: Arabic
AI: 

Al-Fez Restaurant


#### ConversationChain

In [73]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [74]:
convo.run("Who won the first cricket world cup?")

' The first Cricket World Cup was won by the West Indies in 1975.'

In [75]:
convo.run("How much is 5+5?")

' 10.'

In [76]:
convo.run("Who was the captain ofthe winning team?")

' The captain of the winning team in the first Cricket World Cup was Clive Lloyd.'

In [77]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first Cricket World Cup was won by the West Indies in 1975.
Human: How much is 5+5?
AI:  10.
Human: Who was the captain ofthe winning team?
AI:  The captain of the winning team in the first Cricket World Cup was Clive Lloyd.


#### ConversationBufferWindowMemory

In [82]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.run("Who won the first cricket world cup?")

' The first cricket world cup was held in 1975 and the winning team was the West Indies.'

In [83]:
convo.run("How much is 5+5?")

' The answer is 10.'

In [84]:
convo.run("Who was the captain of the winning team?")

' The captain of the winning team in the 1975 Cricket World Cup was Clive Lloyd.'

In [85]:
print(convo.memory.buffer)

Human: How much is 5+5?
AI:  The answer is 10.
Human: Who was the captain of the winning team?
AI:  The captain of the winning team in the 1975 Cricket World Cup was Clive Lloyd.
